# DATA PREPROCESS
A notebook to preprocess the data.

# Prepare

In [1]:
!pip install pyvi
!pip install rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.9/966.9 kB 53.2 MB/s eta 0:00:00


In [2]:
import os
import json
import random
import pandas as pd
from pyvi.ViTokenizer import tokenize
from rank_bm25 import BM25Plus
from preprocess import build_corpus, build_short_corpus, build_qa, build_biencoder_data, build_general_data, build_short_data

# Define parameters

In [3]:
# Number of hard negatives for each questions generated by BM25+
no_hneg = 7
# Number of retrieved BM25+ documents for each question 
no_search = 100
# Limited token length for a passage
size = 200
limited_length = size + 10
overlap = 64
head = False

In [4]:
with open("/kaggle/input/ltr2021/law_dict.json", 'r') as input:
    law_dict = json.load(input)

# Build corpus

- Sliding corpus

In [5]:
dscorpus = build_short_corpus(tokenize, "/kaggle/input/ltr2021/clean_legal_corpus.json", law_dict, head, size, overlap)

In [6]:
dscorpus

,id,law_id,article_id,text_id,title,text,processed_text,sub_id,tokenized_text,bm25text,len
0,0,01/2009/tt-bnn,1,01/2009/tt-bnn_1,Điều 1. Phạm vi áp dụng,"Thông tư này hướng dẫn tuần tra, canh gác bảo ...","tuần tra, canh gác bảo vệ đê điều trong mùa lũ...",0,"tuần_tra , canh_gác bảo_vệ đê_điều trong mùa l...",tuần_tra canh_gác bảo_vệ đê_điều mùa lũ phạm_v...,36
1,1,01/2009/tt-bnn,2,01/2009/tt-bnn_2,Điều 2. Tổ chức lực lượng,"1. Hàng năm trước mùa mưa, lũ, Ủy ban nhân dân...","tuần tra, canh gác bảo vệ đê điều trong mùa lũ...",1,"tuần_tra , canh_gác bảo_vệ đê_điều trong mùa l...",tuần_tra canh_gác bảo_vệ đê_điều mùa lũ tổ_chứ...,182
2,2,01/2009/tt-bnn,3,01/2009/tt-bnn_3,Điều 3. Tiêu chuẩn của các thành viên thuộc lự...,"1. Là người khoẻ mạnh, tháo vát, đủ khả năng đ...","tuần tra, canh gác bảo vệ đê điều trong mùa lũ...",2,"tuần_tra , canh_gác bảo_vệ đê_điều trong mùa l...",tuần_tra canh_gác bảo_vệ đê_điều mùa lũ tiêu_c...,72
3,3,01/2009/tt-bnn,4,01/2009/tt-bnn_4,"Điều 4. Nhiệm vụ của lực lượng tuần tra, canh ...",1. Chấp hành sự phân công của Ban chỉ huy phòn...,"tuần tra, canh gác bảo vệ đê điều trong mùa lũ...",3,"tuần_tra , canh_gác bảo_vệ đê_điều trong mùa l...",tuần_tra canh_gác bảo_vệ đê_điều mùa lũ nhiệm_...,171
4,4,01/2009/tt-bnn,5,01/2009/tt-bnn_5,"Điều 5. Phù hiệu của lực lượng tuần tra, canh ...","Phù hiệu của lực lượng tuần tra, canh gác đê l...","tuần tra, canh gác bảo vệ đê điều trong mùa lũ...",4,"tuần_tra , canh_gác bảo_vệ đê_điều trong mùa l...",tuần_tra canh_gác bảo_vệ đê_điều mùa lũ phù_hi...,53
...,...,...,...,...,...,...,...,...,...,...,...
108679,60793,99/2020/nđ-cp,63,99/2020/nđ-cp_63,Điều 63. Phân định thẩm quyền xử phạt của Chủ ...,1. Phân định thẩm quyền xử phạt của Chủ tịch Ủ...,quy đi nh xử phạt vi phạm hành chính trong lĩn...,108679,quy đi nh xử_phạt vi_phạm hành_chính trong lĩn...,quy đi nh xử_phạt vi_phạm hành_chính lĩnh_vực ...,240
108680,60793,99/2020/nđ-cp,63,99/2020/nđ-cp_63,Điều 63. Phân định thẩm quyền xử phạt của Chủ ...,1. Phân định thẩm quyền xử phạt của Chủ tịch Ủ...,quy đi nh xử phạt vi phạm hành chính trong lĩn...,108680,quy đi nh xử_phạt vi_phạm hành_chính trong lĩn...,quy đi nh xử_phạt vi_phạm hành_chính lĩnh_vực ...,240
108681,60793,99/2020/nđ-cp,63,99/2020/nđ-cp_63,Điều 63. Phân định thẩm quyền xử phạt của Chủ ...,1. Phân định thẩm quyền xử phạt của Chủ tịch Ủ...,quy đi nh xử phạt vi phạm hành chính trong lĩn...,108681,quy đi nh xử_phạt vi_phạm hành_chính trong lĩn...,quy đi nh xử_phạt vi_phạm hành_chính lĩnh_vực ...,240
108682,60793,99/2020/nđ-cp,63,99/2020/nđ-cp_63,Điều 63. Phân định thẩm quyền xử phạt của Chủ ...,1. Phân định thẩm quyền xử phạt của Chủ tịch Ủ...,quy đi nh xử phạt vi phạm hành chính trong lĩn...,108682,quy đi nh xử_phạt vi_phạm hành_chính trong lĩn...,quy đi nh xử_phạt vi_phạm hành_chính lĩnh_vực ...,108


In [7]:
dscorpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108684 entries, 0 to 108683
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   id              108684 non-null  int64 
 1   law_id          108684 non-null  object
 2   article_id      108684 non-null  object
 3   text_id         108684 non-null  object
 4   title           108684 non-null  object
 5   text            108684 non-null  object
 6   processed_text  108684 non-null  object
 7   sub_id          108684 non-null  int64 
 8   tokenized_text  108684 non-null  object
 9   bm25text        108684 non-null  object
 10  len             108684 non-null  int64 
dtypes: int64(3), object(8)
memory usage: 9.1+ MB


In [8]:
dscorpus.describe()

,id,sub_id,len
count,108684.000000,108684.000000,108684.000000
mean,30302.456176,54341.500000,160.084870
std,17347.387261,31374.512666,70.551618
min,0.000000,0.000000,3.000000
25%,15645.000000,27170.750000,96.000000
50%,29719.000000,54341.500000,171.000000
75%,45193.250000,81512.250000,224.000000
max,60794.000000,108683.000000,301.000000


In [9]:
dscorpus.to_csv("scorpus.csv", index=False)

- Build corpus

In [10]:
scorpus_ids = dscorpus['id'].tolist()

In [11]:
dcorpus = build_corpus(tokenize, "/kaggle/input/ltr2021/clean_legal_corpus.json", law_dict, scorpus_ids, head)

In [12]:
dcorpus

,id,law_id,article_id,text_id,title,text,processed_text,sub_id,tokenized_text,bm25text,len
0,0,01/2009/tt-bnn,1,01/2009/tt-bnn_1,Điều 1. Phạm vi áp dụng,"Thông tư này hướng dẫn tuần tra, canh gác bảo ...","tuần tra, canh gác bảo vệ đê điều trong mùa lũ...",[0],"tuần_tra , canh_gác bảo_vệ đê_điều trong mùa l...",tuần_tra canh_gác bảo_vệ đê_điều mùa lũ phạm_v...,36
1,1,01/2009/tt-bnn,2,01/2009/tt-bnn_2,Điều 2. Tổ chức lực lượng,"1. Hàng năm trước mùa mưa, lũ, Ủy ban nhân dân...","tuần tra, canh gác bảo vệ đê điều trong mùa lũ...",[1],"tuần_tra , canh_gác bảo_vệ đê_điều trong mùa l...",tuần_tra canh_gác bảo_vệ đê_điều mùa lũ tổ_chứ...,182
2,2,01/2009/tt-bnn,3,01/2009/tt-bnn_3,Điều 3. Tiêu chuẩn của các thành viên thuộc lự...,"1. Là người khoẻ mạnh, tháo vát, đủ khả năng đ...","tuần tra, canh gác bảo vệ đê điều trong mùa lũ...",[2],"tuần_tra , canh_gác bảo_vệ đê_điều trong mùa l...",tuần_tra canh_gác bảo_vệ đê_điều mùa lũ tiêu_c...,72
3,3,01/2009/tt-bnn,4,01/2009/tt-bnn_4,"Điều 4. Nhiệm vụ của lực lượng tuần tra, canh ...",1. Chấp hành sự phân công của Ban chỉ huy phòn...,"tuần tra, canh gác bảo vệ đê điều trong mùa lũ...",[3],"tuần_tra , canh_gác bảo_vệ đê_điều trong mùa l...",tuần_tra canh_gác bảo_vệ đê_điều mùa lũ nhiệm_...,171
4,4,01/2009/tt-bnn,5,01/2009/tt-bnn_5,"Điều 5. Phù hiệu của lực lượng tuần tra, canh ...","Phù hiệu của lực lượng tuần tra, canh gác đê l...","tuần tra, canh gác bảo vệ đê điều trong mùa lũ...",[4],"tuần_tra , canh_gác bảo_vệ đê_điều trong mùa l...",tuần_tra canh_gác bảo_vệ đê_điều mùa lũ phù_hi...,53
...,...,...,...,...,...,...,...,...,...,...,...
60790,60790,99/2020/nđ-cp,60,99/2020/nđ-cp_60,Điều 60. Thẩm quyền xử phạt của Hải quan,"1. Chi cục trưởng Chi cục Hải quan, Chi cục tr...",quy đi nh xử phạt vi phạm hành chính trong lĩn...,"[108664, 108665]",quy đi nh xử_phạt vi_phạm hành_chính trong lĩn...,quy đi nh xử_phạt vi_phạm hành_chính lĩnh_vực ...,255
60791,60791,99/2020/nđ-cp,61,99/2020/nđ-cp_61,Điều 61. Thẩm quyền xử phạt của Quản lý thị tr...,1. Kiểm soát viên thị trường đang thi hành côn...,quy đi nh xử phạt vi phạm hành chính trong lĩn...,"[108666, 108667]",quy đi nh xử_phạt vi_phạm hành_chính trong lĩn...,quy đi nh xử_phạt vi_phạm hành_chính lĩnh_vực ...,239
60792,60792,99/2020/nđ-cp,62,99/2020/nđ-cp_62,Điều 62. Thẩm quyền xử phạt của Thanh tra,"1. Thanh tra viên, người được giao thực hiện n...",quy đi nh xử phạt vi phạm hành chính trong lĩn...,"[108668, 108669, 108670]",quy đi nh xử_phạt vi_phạm hành_chính trong lĩn...,quy đi nh xử_phạt vi_phạm hành_chính lĩnh_vực ...,420
60793,60793,99/2020/nđ-cp,63,99/2020/nđ-cp_63,Điều 63. Phân định thẩm quyền xử phạt của Chủ ...,1. Phân định thẩm quyền xử phạt của Chủ tịch Ủ...,quy đi nh xử phạt vi phạm hành chính trong lĩn...,"[108671, 108672, 108673, 108674, 108675, 10867...",quy đi nh xử_phạt vi_phạm hành_chính trong lĩn...,quy đi nh xử_phạt vi_phạm hành_chính lĩnh_vực ...,1852


- 61425 legal documents

In [13]:
dcorpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60795 entries, 0 to 60794
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              60795 non-null  int64 
 1   law_id          60795 non-null  object
 2   article_id      60795 non-null  object
 3   text_id         60795 non-null  object
 4   title           60795 non-null  object
 5   text            60795 non-null  object
 6   processed_text  60795 non-null  object
 7   sub_id          60795 non-null  object
 8   tokenized_text  60795 non-null  object
 9   bm25text        60795 non-null  object
 10  len             60795 non-null  int64 
dtypes: int64(2), object(9)
memory usage: 5.1+ MB


In [14]:
dcorpus.describe()

,id,len
count,60795.000000,60795.000000
mean,30397.000000,227.380064
std,17550.149145,405.256854
min,0.000000,9.000000
25%,15198.500000,91.000000
50%,30397.000000,153.000000
75%,45595.500000,268.000000
max,60794.000000,44574.000000


- 75% of the documents (after segemented) have length <= 256

In [15]:
#os.makedirs("data/processed")

In [16]:
#Save corpus
dcorpus.to_csv("corpus.csv", index=False)

# Build q-a data

In [17]:
text_ids = dcorpus["text_id"].tolist()

In [18]:
dqa = build_qa(tokenize, dcorpus, "/kaggle/input/ltr2021/train_question_answer.json")

In [19]:
dqa

,question,processed_question,tokenized_question,bm25_question,ques_len,no_ans,ans_text_id,ans_id,ans_title,ans_text,ans_len,ans_sub_id
0,Công an xã xử phạt lỗi không mang bằng lái xe ...,công an xã xử phạt lỗi không mang bằng lái xe ...,công_an xã xử_phạt lỗi không mang bằng lái_xe ...,công_an xã xử_phạt lỗi mang bằng lái_xe đúng,11,1,47/2011/tt-bca_7,45073,Điều 7. Nhiệm vụ của lực lượng Cảnh sát khác v...,"1. Bố trí lực lượng tham gia tuần tra, kiểm so...",229,"[81289, 81290]"
1,Phải thực hiện thao tác nạp mẫu vào bình chứa ...,phải thực hiện thao tác nạp mẫu vào bình chứa ...,phải thực_hiện thao_tác nạp mẫu vào bình chứa ...,thực_hiện thao_tác nạp mẫu bình chứa xử_lý mẫu...,16,1,41/2020/tt-bca_11,42683,"Điều 11. Tiến hành lấy, bảo quản mẫu",1. Chuẩn bị cho việc lấy mẫu\na) Dọn sạch khu ...,673,"[77284, 77285, 77286, 77287, 77288]"
2,Trình tự đánh giá chất lượng hàng năm đối với ...,trình tự đánh giá chất lượng hàng năm đối với ...,trình_tự đánh_giá chất_lượng hàng năm đối_với ...,trình_tự đánh_giá chất_lượng hàng đối_với kiểm...,12,1,159/2020/nđ-cp_13,22304,"Điều 13. Trình tự, thủ tục đánh giá",1. Đối với người quản lý doanh nghiệp nhà nước...,350,"[40405, 40406, 40407]"
3,Tử tù chết sau khi tiêm thuốc độc mà không có ...,tử tù chết sau khi tiêm thuốc độc mà không có ...,tử_tù chết sau khi tiêm thuốc_độc mà không có ...,tử_tù chết tiêm thuốc_độc thân_nhân nhận xác h...,18,2,"53/2010/qh12_60, 82/2011/nđ-cp_9","47537, 55143","Điều 60. Giải quyết việc xin nhận tử thi, hài ...",1. Việc giải quyết nhận tử thi được thực hiện ...,"567, 93","[85350, 85351, 85352, 85353, 98757]"
4,Trách nhiệm của Bộ Khoa học và Công nghệ về qu...,trách nhiệm của bộ khoa học và công nghệ về qu...,trách_nhiệm của bộ khoa_học và công_nghệ về qu...,trách_nhiệm khoa_học công_nghệ quản_lý phát_tr...,16,1,63/2020/nđ-cp_20,51235,"Điều 20. Trách nhiệm của các bộ, cơ quan ngang bộ",1. Bộ Quốc phòng có trách nhiệm phối hợp với B...,591,"[91816, 91817, 91818, 91819]"
...,...,...,...,...,...,...,...,...,...,...,...,...
3191,Nội dung kiểm tra công tác nghiệm thu công trì...,nội dung kiểm tra công tác nghiệm thu công trì...,nội_dung kiểm_tra công_tác nghiệm_thu công_trì...,nội_dung kiểm_tra công_tác nghiệm_thu công_trì...,13,1,06/2021/nđ-cp_24,6882,Điều 24. Kiểm tra công tác nghiệm thu công trì...,1. Công trình xây dựng phải được cơ quan nhà n...,924,"[11318, 11319, 11320, 11321, 11322, 11323]"
3192,Mức phạt đối với hành vi chuyển mục đích sử dụ...,mức phạt đối với hành vi chuyển mục đích sử dụ...,mức phạt đối_với hành_vi chuyển mục_đích sử_dụ...,mức phạt đối_với hành_vi chuyển mục_đích sử_dụ...,27,1,35/2019/nđ-cp_12,38908,Điều 12. Vi phạm quy định về chuyển mục đích s...,Hành vi chuyển mục đích sử dụng rừng sang mục ...,378,"[70518, 70519, 70520]"
3193,Trích xuất theo thi hành án hình sự được hiểu ...,trích xuất theo thi hành án hình sự được hiểu ...,trích xuất theo thi_hành án hình_sự được hiểu ...,trích xuất thi_hành án hình_sự hiểu thế_nào,9,1,41/2019/qh14_3,42467,Điều 3. Giải thích từ ngữ,"Trong Luật này, các từ ngữ dưới đây được hiểu ...",1073,"[76831, 76832, 76833, 76834, 76835, 76836, 76837]"
3194,"Chạy ô tô để dầu, nhờn rơi xuống đường phạt ba...",chạy ô tô để dầu nhờn rơi xuống đường phạt bao...,chạy ô_tô để dầu_nhờn rơi xuống_đường phạt bao...,chạy ô_tô dầu_nhờn rơi xuống_đường phạt bao_nhiêu,8,1,100/2019/nđ-cp_20,10307,"Điều 20. Xử phạt người điều khiển xe ô tô, máy...",1. Phạt tiền từ 100.000 đồng đến 200.000 đồng ...,318,"[17361, 17362]"


- 3196 questions

In [20]:
dqa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3196 entries, 0 to 3195
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   question            3196 non-null   object
 1   processed_question  3196 non-null   object
 2   tokenized_question  3196 non-null   object
 3   bm25_question       3196 non-null   object
 4   ques_len            3196 non-null   int64 
 5   no_ans              3196 non-null   int64 
 6   ans_text_id         3196 non-null   object
 7   ans_id              3196 non-null   object
 8   ans_title           3196 non-null   object
 9   ans_text            3196 non-null   object
 10  ans_len             3196 non-null   object
 11  ans_sub_id          3196 non-null   object
dtypes: int64(2), object(10)
memory usage: 299.8+ KB


In [21]:
dqa.describe()

,ques_len,no_ans
count,3196.000000,3196.000000
mean,12.683667,1.031915
std,4.014262,0.191154
min,2.000000,1.000000
25%,10.000000,1.000000
50%,12.000000,1.000000
75%,15.000000,1.000000
max,32.000000,3.000000


In [22]:
dqa.loc[dqa['no_ans'] == 2]

,question,processed_question,tokenized_question,bm25_question,ques_len,no_ans,ans_text_id,ans_id,ans_title,ans_text,ans_len,ans_sub_id
3,Tử tù chết sau khi tiêm thuốc độc mà không có ...,tử tù chết sau khi tiêm thuốc độc mà không có ...,tử_tù chết sau khi tiêm thuốc_độc mà không có ...,tử_tù chết tiêm thuốc_độc thân_nhân nhận xác h...,18,2,"53/2010/qh12_60, 82/2011/nđ-cp_9","47537, 55143","Điều 60. Giải quyết việc xin nhận tử thi, hài ...",1. Việc giải quyết nhận tử thi được thực hiện ...,"567, 93","[85350, 85351, 85352, 85353, 98757]"
34,Nếu sử dụng NLĐ chưa đủ 18 tuổi sản xuất bia p...,nếu sử dụng NLĐ chưa đủ 18 tuổi sản xuất bia p...,nếu sử_dụng NLĐ chưa đủ 18 tuổi sản_xuất bia p...,sử_dụng nlđ chưa đủ 18 tuổi sản_xuất bia phạt ...,11,2,"98/2020/nđ-cp_4, 98/2020/nđ-cp_30","60177, 60203",Điều 4. Quy định các hình thức xử phạt vi phạm...,1. Các hình thức xử phạt chính:\na) Cảnh cáo;\...,"385, 185","[107272, 107273, 107274, 107346]"
61,Sử dụng người chưa đủ 18 tuổi bán thuốc lá bị ...,sử dụng người chưa đủ 18 tuổi bán thuốc lá bị ...,sử_dụng người chưa đủ 18 tuổi bán thuốc_lá bị ...,sử_dụng chưa đủ 18 tuổi bán thuốc_lá phạt bao_...,11,2,"98/2020/nđ-cp_4, 98/2020/nđ-cp_23","60177, 60196",Điều 4. Quy định các hình thức xử phạt vi phạm...,1. Các hình thức xử phạt chính:\na) Cảnh cáo;\...,"385, 292","[107272, 107273, 107274, 107333, 107334]"
73,Công an hiệu được quy định như thế nào theo qu...,công an hiệu được quy định như thế nào theo qu...,công_an hiệu được quy_định như thế_nào theo qu...,công_an hiệu quy_định thế_nào quy_định mới,9,2,"160/2007/nđ-cp_2, 29/2016/nđ-cp_1","23424, 35871","Điều 2. Công an hiệu, Điều 1. Sửa đổi, bổ sung...","1. Công an hiệu hình tròn, đường kính 36 mm, ở...","131, 578","[42564, 64809, 64810, 64811, 64812]"
84,Lễ phục của sĩ quan Công an nhân dân được quy ...,lễ phục của sĩ quan công an nhân dân được quy ...,lễ_phục của sĩ_quan công_an nhân_dân được quy_...,lễ_phục sĩ_quan công_an nhân_dân quy_định thế_nào,9,2,"160/2007/nđ-cp_6, 29/2016/nđ-cp_1","23428, 35871",Điều 6. Trang phục của lực lượng Công an nhân ...,Trang phục của lực lượng Công an nhân dân gồm:...,"242, 578","[42572, 42573, 64809, 64810, 64811, 64812]"
...,...,...,...,...,...,...,...,...,...,...,...,...
2819,Có phải bồi thường khi gây tai nạn giao thông ...,có phải bồi thường khi gây tai nạn giao thông ...,có phải bồi_thường_khi gây tai_nạn giao_thông ...,bồi_thường_khi gây tai_nạn giao_thông lỗi mình,12,2,"91/2015/qh13_584, 91/2015/qh13_585","58123, 58124",Điều 584. Căn cứ phát sinh trách nhiệm bồi thư...,"1. Người nào có hành vi xâm phạm tính mạng, sứ...","146, 197","[103655, 103656]"
2901,Đang tạm giam có được khi không phải người thâ...,đang tạm giam có được khi không phải người thâ...,đang tạm giam có được khi không phải người_thâ...,đang tạm giam người_thân,10,2,"94/2015/qh13_3, 94/2015/qh13_22","59479, 59498","Điều 3. Giải thích từ ngữ, Điều 22. Việc gặp t...","Trong Luật này, các từ ngữ dưới đây được hiểu ...","484, 665","[105975, 105976, 105977, 105978, 106024, 10602..."
2986,"Xây dựng, phê duyệt kế hoạch tiến hành thanh t...",xây dựng phê duyệt kế hoạch tiến hành thanh tr...,xây_dựng phê_duyệt kế_hoạch tiến_hành thanh_tr...,xây_dựng phê_duyệt kế_hoạch tiến_hành thanh_tr...,9,2,"07/2012/nđ-cp_18, 05/2014/tt-ttcp_18","7007, 4955",Điều 18. Xây dựng và phê duyệt kế hoạch tiến h...,1. Trưởng đoàn thanh tra có trách nhiệm xây dự...,"139, 130","[11526, 8140]"
3120,Quản lý và sử dụng phí tham quan Bảo tàng Dân ...,quản lý và sử dụng phí tham quan bảo tàng dân ...,quản_lý và sử_dụng phí tham_quan bảo_tàng dân_...,quản_lý sử_dụng phí tham_quan bảo_tàng dân_tộc...,13,2,"120/2016/nđ-cp_5, 160/2016/tt-btc_7","15746, 23472","Điều 5. Xác định tỷ lệ để lại và quản lý, sử d...",1. Số tiền phí được khấu trừ hoặc để lại cho t...,"721, 81","[27413, 27414, 27415, 27416, 27417, 42649]"


In [23]:
dqa.loc[dqa['no_ans'] == 3]

,question,processed_question,tokenized_question,bm25_question,ques_len,no_ans,ans_text_id,ans_id,ans_title,ans_text,ans_len,ans_sub_id
206,"Cục Xuất bản, In và Phát hành có trách nhiệm g...",cục xuất bản in và phát hành có trách nhiệm gì...,cục xuất_bản in và phát_hành có trách_nhiệm gì...,cục xuất_bản in phát_hành trách_nhiệm gì thẩm_...,17,3,"195/2013/nđ-cp_17, 01/2020/tt-btttt_24, 195/20...","27499, 1475, 27500","Điều 17. Điều kiện hoạt động xuất bản, phát hà...","1. Điều kiện về thiết bị, công nghệ để xuất bả...","394, 173, 284","[50327, 50328, 50329, 2469, 50330, 50331]"
258,Tổ chức lựa chọn nhà thầu và bàn giao mặt bằng...,tổ chức lựa chọn nhà thầu và bàn giao mặt bằng...,tổ_chức lựa_chọn nhà_thầu và bàn_giao mặt_bằng...,tổ_chức lựa_chọn nhà_thầu bàn_giao mặt_bằng th...,16,3,"159/2018/nđ-cp_18, 159/2018/nđ-cp_17, 33/2019/...","22260, 22259, 37968","Điều 18. Bàn giao mặt bằng thi công, Điều 17. ...",1. Công tác đo đạc bàn giao mặt bằng được triể...,"366, 345, 50","[40344, 40345, 40346, 40341, 40342, 40343, 68749]"
492,Trách nhiệm của Chủ dự án sau khi báo cáo ĐTM ...,trách nhiệm của chủ dự án sau khi báo cáo ĐTM ...,trách_nhiệm của chủ dự_án sau khi báo_cáo ĐTM ...,trách_nhiệm chủ dự_án báo_cáo đtm phê_duyệt qu...,14,3,"18/2015/nđ-cp_16, 18/2016/tt-bnnptnt_17, 18/20...","26825, 26879, 26826",Điều 16. Trách nhiệm của chủ dự án sau khi báo...,"1. Trong trường hợp cần thiết, điều chỉnh nội ...","357, 85, 219","[48742, 48743, 48744, 48818, 48745, 48746]"
764,Cựu thanh niên xung phong được hưởng chế độ BH...,cựu thanh niên xung phong được hưởng chế độ BH...,cựu_thanh_niên xung_phong được hưởng chế_độ BH...,cựu_thanh_niên xung_phong hưởng chế_độ bhyt th...,8,3,"146/2018/nđ-cp_14, 146/2018/nđ-cp_11, 146/2018...","20054, 20051, 20043",Điều 14. Mức hưởng bảo hiểm y tế đối với các t...,1. Người tham gia bảo hiểm y tế khi đi khám bệ...,"565, 152, 1321","[36035, 36036, 36037, 36038, 36028, 36003, 360..."
919,Quản lý và sử dụng phí bảo vệ môi trường đối v...,quản lý và sử dụng phí bảo vệ môi trường đối v...,quản_lý và sử_dụng phí bảo_vệ môi_trường đối_v...,quản_lý sử_dụng phí bảo_vệ môi_trường đối_với ...,13,3,"120/2016/nđ-cp_5, 53/2020/nđ-cp_9, 120/2016/nđ...","15746, 47901, 15745","Điều 5. Xác định tỷ lệ để lại và quản lý, sử d...",1. Số tiền phí được khấu trừ hoặc để lại cho t...,"721, 441, 244","[27413, 27414, 27415, 27416, 27417, 85958, 859..."
1631,"Trình thẩm định, phê duyệt chương trình quản l...",trình thẩm định phê duyệt chương trình quản lý...,trình thẩm_định phê_duyệt chương_trình quản_lý...,trình thẩm_định phê_duyệt chương_trình quản_lý...,13,3,"40/2016/nđ-cp_17, 40/2016/nđ-cp_16, 49/2017/tt...","41746, 41745, 46106",Điều 17. Phê duyệt chương trình quản lý tổng h...,1. Hồ sơ phê duyệt chương trình gồm:\na) Tờ tr...,"149, 310, 69","[75359, 75357, 75358, 83004]"
2479,Chi phí và lưu trữ hồ sơ trong lựa chọn nhà th...,chi phí và lưu trữ hồ sơ trong lựa chọn nhà th...,chi_phí và lưu_trữ hồ_sơ trong lựa_chọn nhà_th...,chi_phí lưu_trữ hồ_sơ lựa_chọn nhà_thầu cơ_sở ...,15,3,"63/2014/nđ-cp_10, 15/2019/tt-byt_6, 63/2014/nđ...","50913, 23098, 50912",Điều 10. Lưu trữ hồ sơ trong quá trình lựa chọ...,1. Toàn bộ hồ sơ liên quan đến quá trình lựa c...,"208, 62, 480","[91252, 41857, 91249, 91250, 91251]"
3026,Con làm bạn gãy tay thì phụ huynh có phải bồi ...,con làm bạn gãy tay thì phụ huynh có phải bồi ...,con làm_bạn gãy tay thì phụ_huynh có phải bồi_...,con làm_bạn gãy tay phụ_huynh bồi_thường,9,3,"91/2015/qh13_584, 91/2015/qh13_586, 91/2015/qh...","58123, 58125, 58124",Điều 584. Căn cứ phát sinh trách nhiệm bồi thư...,"1. Người nào có hành vi xâm phạm tính mạng, sứ...","146, 243, 197","[103655, 103657, 103658, 103656]"
3032,"Việc quản lý phí, lệ phí trong thú y được quy ...",việc quản lý phí lệ phí trong thú y được quy đ...,việc quản_lý phí lệ_phí trong thú_y được quy_đ...,quản_lý phí lệ_phí thú_y quy_định thế_nào,10,3,"120/2016/nđ-cp_5, 101/2020/tt-btc_6, 120/2016/...","15746, 11055, 15745","Điều 5. Xác định tỷ lệ để lại và quản lý,

Of all 3196 questions:

- 3103 have 1 relevant article (97.09%)
- 84 have 2 relevant articles (2.63%)
- 9 have 3 relevant articles (0.28%)

In [24]:
dqa.to_csv("qa.csv", index=False)

- Build qa split data (1 question - 1 answer) for biencoder training 

In [25]:
dqa_split = build_qa(tokenize, dcorpus, "/kaggle/input/ltr2021/train_question_answer.json", split = True)

In [26]:
dqa_split

,question,processed_question,tokenized_question,bm25_question,ques_len,ans_text_id,ans_id,ans_title,ans_text,ans_len,ans_sub_id
0,Công an xã xử phạt lỗi không mang bằng lái xe ...,công an xã xử phạt lỗi không mang bằng lái xe ...,công_an xã xử_phạt lỗi không mang bằng lái_xe ...,công_an xã xử_phạt lỗi mang bằng lái_xe đúng,11,47/2011/tt-bca_7,45073,Điều 7. Nhiệm vụ của lực lượng Cảnh sát khác v...,"1. Bố trí lực lượng tham gia tuần tra, kiểm so...",229,"[81289, 81290]"
1,Phải thực hiện thao tác nạp mẫu vào bình chứa ...,phải thực hiện thao tác nạp mẫu vào bình chứa ...,phải thực_hiện thao_tác nạp mẫu vào bình chứa ...,thực_hiện thao_tác nạp mẫu bình chứa xử_lý mẫu...,16,41/2020/tt-bca_11,42683,"Điều 11. Tiến hành lấy, bảo quản mẫu",1. Chuẩn bị cho việc lấy mẫu\na) Dọn sạch khu ...,673,"[77284, 77285, 77286, 77287, 77288]"
2,Trình tự đánh giá chất lượng hàng năm đối với ...,trình tự đánh giá chất lượng hàng năm đối với ...,trình_tự đánh_giá chất_lượng hàng năm đối_với ...,trình_tự đánh_giá chất_lượng hàng đối_với kiểm...,12,159/2020/nđ-cp_13,22304,"Điều 13. Trình tự, thủ tục đánh giá",1. Đối với người quản lý doanh nghiệp nhà nước...,350,"[40405, 40406, 40407]"
3,Tử tù chết sau khi tiêm thuốc độc mà không có ...,tử tù chết sau khi tiêm thuốc độc mà không có ...,tử_tù chết sau khi tiêm thuốc_độc mà không có ...,tử_tù chết tiêm thuốc_độc thân_nhân nhận xác h...,18,53/2010/qh12_60,47537,"Điều 60. Giải quyết việc xin nhận tử thi, hài ...",1. Việc giải quyết nhận tử thi được thực hiện ...,567,"[85350, 85351, 85352, 85353]"
4,Tử tù chết sau khi tiêm thuốc độc mà không có ...,tử tù chết sau khi tiêm thuốc độc mà không có ...,tử_tù chết sau khi tiêm thuốc_độc mà không có ...,tử_tù chết tiêm thuốc_độc thân_nhân nhận xác h...,18,82/2011/nđ-cp_9,55143,Điều 9. Chi phí mai táng,Chi phí mai táng người bị thi hành án tử hình ...,93,[98757]
...,...,...,...,...,...,...,...,...,...,...,...
3293,Nội dung kiểm tra công tác nghiệm thu công trì...,nội dung kiểm tra công tác nghiệm thu công trì...,nội_dung kiểm_tra công_tác nghiệm_thu công_trì...,nội_dung kiểm_tra công_tác nghiệm_thu công_trì...,13,06/2021/nđ-cp_24,6882,Điều 24. Kiểm tra công tác nghiệm thu công trì...,1. Công trình xây dựng phải được cơ quan nhà n...,924,"[11318, 11319, 11320, 11321, 11322, 11323]"
3294,Mức phạt đối với hành vi chuyển mục đích sử dụ...,mức phạt đối với hành vi chuyển mục đích sử dụ...,mức phạt đối_với hành_vi chuyển mục_đích sử_dụ...,mức phạt đối_với hành_vi chuyển mục_đích sử_dụ...,27,35/2019/nđ-cp_12,38908,Điều 12. Vi phạm quy định về chuyển mục đích s...,Hành vi chuyển mục đích sử dụng rừng sang mục ...,378,"[70518, 70519, 70520]"
3295,Trích xuất theo thi hành án hình sự được hiểu ...,trích xuất theo thi hành án hình sự được hiểu ...,trích xuất theo thi_hành án hình_sự được hiểu ...,trích xuất thi_hành án hình_sự hiểu thế_nào,9,41/2019/qh14_3,42467,Điều 3. Giải thích từ ngữ,"Trong Luật này, các từ ngữ dưới đây được hiểu ...",1073,"[76831, 76832, 76833, 76834, 76835, 76836, 76837]"
3296,"Chạy ô tô để dầu, nhờn rơi xuống đường phạt ba...",chạy ô tô để dầu nhờn rơi xuống đường phạt bao...,chạy ô_tô để dầu_nhờn rơi xuống_đường phạt bao...,chạy ô_tô dầu_nhờn rơi xuống_đường phạt bao_nhiêu,8,100/2019/nđ-cp_20,10307,"Điều 20. Xử phạt người điều khiển xe ô tô, máy...",1. Phạt tiền từ 100.000 đồng đến 200.000 đồng ...,318,"[17361, 17362]"


In [27]:
dqa_split.describe()

,ques_len,ans_id,ans_len
count,3298.000000,3298.000000,3298.000000
mean,12.681928,29261.943299,520.238933
std,3.994436,17978.761287,1647.235300
min,2.000000,108.000000,30.000000
25%,10.000000,10317.250000,119.000000
50%,12.000000,28734.000000,251.000000
75%,15.000000,45697.000000,500.000000
max,32.000000,60734.000000,44117.000000


In [28]:
dqa_split.to_csv("qa_split.csv", index=False)

# Split into 3 datasets: Train, Validation, Test

In [29]:
questions = dqa['question'].tolist()
seed = 0.5 
random.seed(seed)
random.shuffle(questions)
train_ques = questions[0:2400]
val_ques = questions[2400:2750]
test_ques = questions[2750:]

- Using BM25+ to get hard negatives and compare performace with dual-encoder retriever

In [30]:
bm25texts = dcorpus['bm25text'].tolist()
split_bm25texts = [text.split(" ") for text in bm25texts]
bm25 = BM25Plus(split_bm25texts)

In [31]:
ids = range(bm25.corpus_size)

In [32]:
query = "tử tù chết sau khi tiêm thuốc độc mà không có thân nhân nhận xác thì được hỗ trợ mai táng".split(" ")
doc_ids = bm25.get_top_n(query, ids, n=10)

In [33]:
doc_ids

[11642, 43663, 41173, 47597, 46516, 55142, 42604, 53722, 45091, 47526]

## Build data for biencoder training

In [34]:
dval = build_biencoder_data(dqa_split, bm25, val_ques, no_hneg, no_search)
dtrain = build_biencoder_data(dqa_split, bm25, train_ques, no_hneg, no_search)
dtest = build_biencoder_data(dqa_split, bm25, test_ques, no_hneg, no_search)

0.9556786703601108
0.9665726943213854
0.9669603524229075


In [35]:
dtrain.reset_index(drop=True, inplace=True)
dval.reset_index(drop=True, inplace=True)
dtest.reset_index(drop=True, inplace=True)

In [36]:
dtrain_short = build_short_data(dtrain, dcorpus, limited_length)
dtrain_short.to_csv("train_234.csv", index=False)
dval_short = build_short_data(dval, dcorpus, limited_length)
dval_short.to_csv("val_234.csv", index=False)
dtest_short = build_short_data(dtest, dcorpus, limited_length)
dtest_short.to_csv("test_234.csv", index=False)

- Save data

In [37]:
#dir = "data/ltr{}neg".format(no_hneg)
#os.makedirs(dir)
train_dir = "train.csv"
val_dir = "val.csv"
test_dir = "test.csv"

In [38]:
dtrain.to_csv(train_dir, index=False)
dtest.to_csv(test_dir, index=False)
dval.to_csv(val_dir, index=False)

## Build general data for evaluating and training cross-encoder

In [39]:
dtval = build_general_data(dqa, bm25, val_ques, no_hneg, no_search)
dttrain = build_general_data(dqa, bm25, train_ques, no_hneg, no_search)
dttest = build_general_data(dqa, bm25, test_ques, no_hneg, no_search)

0.9571428571428572
0.9683333333333334
0.968609865470852


Testing hit scores (retriving all relevant articles) with top 300 documents retrieved by BM25:

- Train: 96.8750%
- Validation: 96.5714%
- Test: 97.3094%

In [40]:
dtval

,question,processed_question,tokenized_question,bm25_question,ques_len,no_ans,ans_text_id,ans_id,ans_title,ans_text,ans_len,ans_sub_id,neg_ids,search_ids
0,Công an xã xử phạt lỗi không mang bằng lái xe ...,công an xã xử phạt lỗi không mang bằng lái xe ...,công_an xã xử_phạt lỗi không mang bằng lái_xe ...,công_an xã xử_phạt lỗi mang bằng lái_xe đúng,11,1,47/2011/tt-bca_7,45073,Điều 7. Nhiệm vụ của lực lượng Cảnh sát khác v...,"1. Bố trí lực lượng tham gia tuần tra, kiểm so...",229,"[81289, 81290]","[10308, 10324, 59212, 16733, 16735, 10319, 13053]","[10308, 10324, 59212, 16733, 16735, 10319, 130..."
3,Tử tù chết sau khi tiêm thuốc độc mà không có ...,tử tù chết sau khi tiêm thuốc độc mà không có ...,tử_tù chết sau khi tiêm thuốc_độc mà không có ...,tử_tù chết tiêm thuốc_độc thân_nhân nhận xác h...,18,2,"53/2010/qh12_60, 82/2011/nđ-cp_9","47537, 55143","Điều 60. Giải quyết việc xin nhận tử thi, hài ...",1. Việc giải quyết nhận tử thi được thực hiện ...,"567, 93","[85350, 85351, 85352, 85353, 98757]","[11642, 43664, 4918, 4916, 4923, 43679, 4904]","[11642, 43664, 4918, 4916, 4923, 43679, 4904, ..."
4,Trách nhiệm của Bộ Khoa học và Công nghệ về qu...,trách nhiệm của bộ khoa học và công nghệ về qu...,trách_nhiệm của bộ khoa_học và công_nghệ về qu...,trách_nhiệm khoa_học công_nghệ quản_lý phát_tr...,16,1,63/2020/nđ-cp_20,51235,"Điều 20. Trách nhiệm của các bộ, cơ quan ngang bộ",1. Bộ Quốc phòng có trách nhiệm phối hợp với B...,591,"[91816, 91817, 91818, 91819]","[51229, 51234, 44603, 51221, 51230, 51219, 51223]","[51229, 51234, 44603, 51221, 51230, 51219, 512..."
13,Dự án thuộc ngành nghề ưu đãi đầu tư tại địa b...,dự án thuộc ngành nghề ưu đãi đầu tư tại địa b...,dự_án thuộc ngành_nghề ưu_đãi đầu_tư tại địa_b...,dự_án ngành_nghề ưu_đãi đầu_tư địa_bàn khó_khă...,13,1,118/2015/nđ-cp_16,14633,Điều 16. Đối tượng và nguyên tắc áp dụng ưu đã...,1. Đối tượng được hưởng ưu đãi đầu tư theo quy...,364,"[25273, 25274, 25275]","[55246, 55510, 17783, 21273, 55508, 55509, 35823]","[55246, 55510, 17783, 21273, 55508, 55509, 358..."
41,Mức phạt khi đỗ xe ô tô ngược với chiều lưu th...,mức phạt khi đỗ xe ô tô ngược với chiều lưu th...,mức phạt khi đỗ xe ô_tô ngược với chiều lưu_th...,mức phạt đỗ xe ô_tô ngược chiều lưu_thông làn_...,12,1,100/2019/nđ-cp_5,10292,Điều 5. Xử phạt người điều khiển xe ô tô và cá...,1. Phạt tiền từ 200.000 đồng đến 400.000 đồng ...,2697,"[17266, 17267, 17268, 17269, 17270, 17271, 172...","[10294, 10293, 36991, 10295, 56616, 15470, 21224]","[10294, 10293, 36991, 10295, 56616, 15470, 212..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,Chức năng của cơ sở cai nghiện ma túy công lập...,chức năng của cơ sở cai nghiện ma túy công lập...,chức_năng của cơ_sở cai_nghiện ma_túy công_lập...,chức_năng cơ_sở cai_nghiện ma_túy công_lập gồm...,9,1,25/2018/tt-blđtbxh_3,33394,Điều 3. Chức năng,Cơ sở cai nghiện ma túy công lập có các chức n...,60,[60263],"[33880, 33392, 33396, 16108, 33393, 16112, 33398]","[33880, 33392, 33396, 16108, 33393, 16112, 333..."
3151,"Chế độ thông tin, báo cáo của Hội đồng quản lý...",chế độ thông tin báo cáo của hội đồng quản lý ...,chế_độ thông_tin báo_cáo của hội_đồng quản_lý ...,chế_độ thông_tin báo_cáo hội_đồng quản_lý đơn_...,19,1,41/2018/tt-bnnptnt_20,42418,"Điều 20. Chế độ thông tin, báo cáo",Chủ tịch Hội đồng thực hiện chế độ báo cáo địn...,52,[76768],"[42399, 54520, 42413, 42421, 54522, 42417, 42422]","[42399, 54520, 42413, 42421, 54522, 42417, 424..."
3155,Bao lâu thì được thanh toán BHXH 1 lần ?,bao lâu thì được thanh toán BHXH 1 lần,bao_lâu thì được thanh_toán BHXH 1 lần,bao_lâu thanh_toán bhxh lần,7,1,115/2015/nđ-cp_8,14014,Điều 8. Bảo hiểm xã hội một lần,1. Người lao động quy định tại Khoản 1 và Khoả...,286,"[23968, 23969]","[18206, 18199, 46029, 18197, 18195, 11260, 2573]","[18206, 18199, 46029, 18197, 18195, 11260, 257..."
3160,UBND cấp tỉnh có trách nhiệm gì trong việc đán...,UBND cấp tỉnh có trách nhiệm gì trong việc đán...,UBND cấp tỉnh có trách_nhiệm gì trong việc đán...

In [41]:
#Save data
dttrain.to_csv("ttrain.csv", index=False)
dttest.to_csv("ttest.csv", index=False)
dtval.to_csv("tval.csv", index=False)